In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import re
import cptac
import statsmodels.stats.multitest
import operator
import cptac.utils as u


/Users/Lindsey/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
luad = cptac.Luad()
col = cptac.Colon()
en = cptac.Endometrial()

# Proteins to check phospho sites

In [3]:
pik3ca_phospho_sites = ["PIK3CA_phosphoproteomics","AKT1_phosphoproteomics","AKT2_phosphoproteomics","AKT3_phosphoproteomics","MTOR_phosphoproteomics","PDK1_phosphoproteomics","CASP3_phosphoproteomics","KRAS_Mutation","KRAS_Location"]
phospho_sites = ["PIK3CA_phosphoproteomics","MAP3K1_phosphoproteomics","PTPN11_phosphoproteomics","MTOR_phosphoproteomics" "MAP3K2_phosphoproteomics","MAPK1_phosphoproteomics","MAPK1_phosphoproteomics","KRAS_phosphoproteomics","NRAS_phosphoproteomics","HRAS_phosphoproteomics","KRAS_Mutation","KRAS_Location","RAF1_phosphoproteomics","EGFR_phosphoproteomics"]


# Luad

In [4]:
phos = luad.join_omics_to_mutations(omics_df_name = "phosphoproteomics",mutations_genes="KRAS", tissue_type="tumor")

genefilter = phos.columns.get_level_values("Name").isin(phospho_sites)
phos = phos[phos.columns[genefilter]]
phos = u.reduce_multiindex(df=phos, levels_to_drop=["Database_ID","Peptide"])
phos_Kras = cptac.utils.reduce_multiindex(phos, flatten=True)
phos_Kras['KRAS_Location'] = phos_Kras['KRAS_Location'].astype(str)
phos_Kras['KRAS_Mutation'] = phos_Kras['KRAS_Mutation'].astype(str)
pd.set_option('display.max_columns', None)
phos_Kras

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 77 samples for the KRAS gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 1)


Name,EGFR_phosphoproteomics_S1026,EGFR_phosphoproteomics_S1039N1053,EGFR_phosphoproteomics_S1042,EGFR_phosphoproteomics_S1042S1045,EGFR_phosphoproteomics_S1064,EGFR_phosphoproteomics_S1071,EGFR_phosphoproteomics_S1166,EGFR_phosphoproteomics_S991,EGFR_phosphoproteomics_S991S995,EGFR_phosphoproteomics_T1041S1042S1045,EGFR_phosphoproteomics_T693,EGFR_phosphoproteomics_Y1092,EGFR_phosphoproteomics_Y1172,EGFR_phosphoproteomics_Y1197,MAP3K1_phosphoproteomics_S1026,MAP3K1_phosphoproteomics_S1043,MAP3K1_phosphoproteomics_S1455,MAP3K1_phosphoproteomics_S21,MAP3K1_phosphoproteomics_S250,MAP3K1_phosphoproteomics_S266,MAP3K1_phosphoproteomics_S275,MAP3K1_phosphoproteomics_S292,MAP3K1_phosphoproteomics_S35,MAP3K1_phosphoproteomics_S531,MAP3K1_phosphoproteomics_S923,MAPK1_phosphoproteomics_S360,MAPK1_phosphoproteomics_T185Y187,MAPK1_phosphoproteomics_Y187,PIK3CA_phosphoproteomics_S312N319,PTPN11_phosphoproteomics_S36,PTPN11_phosphoproteomics_S562,PTPN11_phosphoproteomics_Y546,PTPN11_phosphoproteomics_Y584,PTPN11_phosphoproteomics_Y62,RAF1_phosphoproteomics_S220,RAF1_phosphoproteomics_S29,RAF1_phosphoproteomics_S316S321,RAF1_phosphoproteomics_S321,RAF1_phosphoproteomics_S519,RAF1_phosphoproteomics_S632,RAF1_phosphoproteomics_T234,RAF1_phosphoproteomics_T330,RAF1_phosphoproteomics_T658,KRAS_Mutation,KRAS_Location
Patient_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C3L-00001,NaN,4.4160,2.5829,NaN,9.0723,NaN,3.3243,1.4256,NaN,NaN,3.4848,NaN,5.9688,8.5140,1.6132,NaN,-2.0372,1.0300,NaN,0.6413,0.7543,NaN,NaN,0.0106,1.0707,-0.2629,-3.4974,-1.1534,0.9781,NaN,-0.4279,-4.6276,-2.9549,-1.8700,0.0197,-1.2393,NaN,-0.6652,0.2615,NaN,NaN,-0.4460,-1.4450,['Wildtype_Tumor'],['No_mutation']
C3L-00009,0.6302,-0.2591,-2.1168,-0.4694,-0.2486,-3.7579,-1.0859,-0.2341,-0.8089,NaN,-0.8547,-2.0647,-1.0692,-1.3295,NaN,NaN,NaN,0.1096,-0.1549,1.1988,1.6424,-0.4569,0.1554,NaN,NaN,0.8489,-1.0359,0.5615,0.6844,NaN,-0.0966,-2.2646,-0.3174,-3.8412,1.6070,-1.3275,NaN,NaN,0.5073,-0.3090,0.7010,NaN,-1.2837,['Wildtype_Tumor'],['No_mutation']
C3L-00080,NaN,NaN,-0.6663,-1.1667,-1.6457,NaN,0.7921,-1.1486,NaN,NaN,-2.0029,NaN,NaN,-2.1724,1.8324,NaN,NaN,0.3032,-1.2440,NaN,0.1650,0.3296,0.3016,NaN,0.3411,NaN,-0.9889,NaN,-0.1461,NaN,-0.0589,-3.9616,-1.9782,-1.9107,-0.1346,-1.7856,NaN,NaN,-0.7996,0.0679,-0.3272,-0.5971,-0.4243,['Wildtype_Tumor'],['No_mutation']
C3L-00083,0.8397,0.1067,-1.1207,-1.0813,-2.2431,NaN,-4.1947,-1.6480,NaN,NaN,-3.6258,NaN,-2.7310,-2.2124,1.3057,NaN,0.3343,-0.5496,-0.0814,NaN,-0.3834,-0.2805,NaN,NaN,NaN,-0.5649,-0.1121,NaN,0.3780,-0.1011,-0.7115,-4.8641,-2.2693,-3.9496,-0.4402,-2.0068,NaN,-0.2762,-0.4709,NaN,0.6493,-0.5956,-0.9938,['Wildtype_Tumor'],['No_mutation']
C3L-00093,-0.2405,NaN,-0.1170,-1.9937,-0.1835,-3.4547,-0.9698,-0.9152,NaN,-1.1052,-1.5019,NaN,-0.7893,0.7454,1.1849,0.9426,NaN,0.9829,NaN,1.4818,1.3892,-0.0362,-0.8130,NaN,-0.0837,-0.2286,-3.6970,NaN,-0.8890,-0.5470,-0.6158,-3.9417,-2.0697,-0.2880,0.9639,-1.1503,0.6218,-0.1051,-0.2357,NaN,0.4651,-1.4521,-0.1384,['Wildtype_Tumor'],['No_mutation']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C3N-02729,-0.0043,NaN,-0.7282,1.5274,2.2272,NaN,2.5832,-0.5842,NaN,NaN,0.2316,NaN,2.8111,1.8433,3.3450,NaN,-0.2003,1.0675,0.3036,1.1695,0.5876,NaN,NaN,NaN,1.7073,NaN,-2.1379,-1.2681,0.2196,0.4476,1.1974,-1.3980,1.1575,6.0525,1.6154,0.9975,-0.0323,-0.9261,-0.5862,0.4556,NaN,-1.0421,-0.2143,['Wildtype_Tumor'],['No_mutation']
X11LU013,NaN,-0.3304,-1.1830,NaN,0.9319,NaN,1.4256,0.0423,NaN,NaN,0.4754,NaN,1.0763,1.3651,-4.0705,NaN,3.9775,0.4208,NaN,1.5504,0.0637,0.4130,NaN,0.0462,-0.4221,-2.7711,-1.5790,-3.1242,-1.3800,NaN,NaN,-7.6662,-4.7318,2.5766,NaN,-2.0316,NaN,NaN,-0.3245,NaN,0.3993,-1.0522,-0.5352,['Wildtype_Tumor'],['No_mutation']
X11LU016,NaN,-0.0832,-0.0211,-0.2712,0.3587,NaN,-3.0185,0.7968,-0.4649,NaN,-0.0418,NaN,-0.3352,-1.1907,-2.8041,0.2985,NaN,-1.1945,0.7705,-1.8639,NaN

In [79]:
hotspot = phos_Kras[(phos_Kras.KRAS_Location.str.contains('G12'))| 
                    phos_Kras.KRAS_Location.str.contains('G13')| 
                    phos_Kras.KRAS_Location.str.contains('Q61')]
wt = phos_Kras[(phos_Kras.KRAS_Mutation.str.contains('Wildtype'))]
hotspot_wt = pd.concat([hotspot, wt])
cols = hotspot_wt.columns.to_list()
cols.remove("KRAS_Mutation")
cols.remove("KRAS_Location")
Kras_phos_pval = u.wrap_ttest(hotspot_wt, 'KRAS_Mutation', cols,return_all=True, pval_return_corrected= True, correction_method= "FDR_bh")
Kras_phos_pval


,Comparison,P_Value
0,PTPN11_phosphoproteomics_Y62,0.000004
1,MAPK1_phosphoproteomics_T185Y187,0.011833
2,RAF1_phosphoproteomics_S519,0.014896
3,MAP3K1_phosphoproteomics_S923,0.014896
4,RAF1_phosphoproteomics_T330,0.014896
5,PIK3CA_phosphoproteomics_S312N319,0.018028
6,PTPN11_phosphoproteomics_Y584,0.021575
7,EGFR_phosphoproteomics_S1166,0.021575
8,EGFR_phosphoproteomics_Y1197,0.021575
9,EGFR_phosphoproteomics_Y1172,0.021575


# Endometrial

In [81]:
phos = en.join_omics_to_mutations(omics_df_name = "phosphoproteomics",mutations_genes="KRAS", tissue_type="tumor")

genefilter = phos.columns.get_level_values("Name").isin(phospho_sites)
phos = phos[phos.columns[genefilter]]
#phos = u.reduce_multiindex(df=phos, levels_to_drop=["Database_ID"])
phos_Kras = cptac.utils.reduce_multiindex(phos, flatten=True)
phos_Kras['KRAS_Location'] = phos_Kras['KRAS_Location'].astype(str)
phos_Kras['KRAS_Mutation'] = phos_Kras['KRAS_Mutation'].astype(str)
phos_Kras.head()

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 64 samples for the KRAS gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 1)


Name,EGFR_phosphoproteomics_S1026,EGFR_phosphoproteomics_S1037,EGFR_phosphoproteomics_S1039,EGFR_phosphoproteomics_S1042,EGFR_phosphoproteomics_S1045,EGFR_phosphoproteomics_S1057,EGFR_phosphoproteomics_S1064,EGFR_phosphoproteomics_S1071,EGFR_phosphoproteomics_S1081,EGFR_phosphoproteomics_S1166,EGFR_phosphoproteomics_S695,EGFR_phosphoproteomics_S991,EGFR_phosphoproteomics_S995,EGFR_phosphoproteomics_T1041,EGFR_phosphoproteomics_T1046,EGFR_phosphoproteomics_T1085,EGFR_phosphoproteomics_T693,EGFR_phosphoproteomics_T993,EGFR_phosphoproteomics_Y1092,EGFR_phosphoproteomics_Y1172,EGFR_phosphoproteomics_Y1197,KRAS_phosphoproteomics_T124,MAP3K1_phosphoproteomics_S1026,MAP3K1_phosphoproteomics_S1043,MAP3K1_phosphoproteomics_S137,MAP3K1_phosphoproteomics_S1455,MAP3K1_phosphoproteomics_S21,MAP3K1_phosphoproteomics_S250,MAP3K1_phosphoproteomics_S252,MAP3K1_phosphoproteomics_S275,MAP3K1_phosphoproteomics_S281,MAP3K1_phosphoproteomics_S292,MAP3K1_phosphoproteomics_S35,MAP3K1_phosphoproteomics_S507,MAP3K1_phosphoproteomics_S531,MAP3K1_phosphoproteomics_S923,MAP3K1_phosphoproteomics_S924,MAP3K1_phosphoproteomics_T1114,MAP3K1_phosphoproteomics_T20,MAP3K1_phosphoproteomics_T285,MAPK1_phosphoproteomics_T181,MAPK1_phosphoproteomics_T185,MAPK1_phosphoproteomics_T190,MAPK1_phosphoproteomics_Y187,NRAS_phosphoproteomics_T122,PIK3CA_phosphoproteomics_S312,PIK3CA_phosphoproteomics_T313,PTPN11_phosphoproteomics_S36,PTPN11_phosphoproteomics_S558,PTPN11_phosphoproteomics_S562,PTPN11_phosphoproteomics_T547,PTPN11_phosphoproteomics_T568,PTPN11_phosphoproteomics_T570,PTPN11_phosphoproteomics_Y546,PTPN11_phosphoproteomics_Y584,PTPN11_phosphoproteomics_Y62,PTPN11_phosphoproteomics_Y63,RAF1_phosphoproteomics_S12,RAF1_phosphoproteomics_S233,RAF1_phosphoproteomics_S243,RAF1_phosphoproteomics_S244,RAF1_phosphoproteomics_S259,RAF1_phosphoproteomics_S29,RAF1_phosphoproteomics_S303,RAF1_phosphoproteomics_S305,RAF1_phosphoproteomics_S307,RAF1_phosphoproteomics_S309,RAF1_phosphoproteomics_S311,RAF1_phosphoproteomics_S315,RAF1_phosphoproteomics_S316,RAF1_phosphoproteomics_S321,RAF1_phosphoproteomics_S641,RAF1_phosphoproteomics_S662,RAF1_phosphoproteomics_T234,RAF1_phosphoproteomics_T239,RAF1_phosphoproteomics_T242,RAF1_phosphoproteomics_T258,RAF1_phosphoproteomics_T260,RAF1_phosphoproteomics_T31,RAF1_phosphoproteomics_T330,RAF1_phosphoproteomics_T658,RAF1_phosphoproteomics_T661,KRAS_Mutation,KRAS_Location
Patient_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C3L-00006,-0.227,NaN,0.0913,-0.03270,-0.116,NaN,0.2970,-1.0200,NaN,0.1410,0.0,-0.324,NaN,-0.11600,NaN,NaN,-0.0745,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.3400,-0.0550,NaN,0.2860,NaN,NaN,NaN,0.736,NaN,NaN,NaN,-0.330,-0.1880,-0.060200,NaN,-0.0782,NaN,-0.00615,0.0731,NaN,NaN,0.0301,NaN,-0.1525,NaN,1.290,NaN,-0.109,NaN,-0.491,NaN,0.217,NaN,NaN,-0.339,NaN,0.0426,0.0426,NaN,-0.00431,NaN,-0.00431,0.019145,0.0388,0.058,NaN,NaN,NaN,NaN,0.263,NaN,-0.340,-0.668,NaN,['Wildtype_Tumor'],['No_mutation']
C3L-00008,NaN,NaN,0.6855,0.00150,-0.245,NaN,NaN,-0.2800,NaN,0.0177,NaN,0.284,NaN,0.19400,NaN,NaN,-0.0316,NaN,NaN,NaN,NaN,NaN,NaN,0.673,NaN,NaN,-0.2030,0.408,0.4080,-0.1865,NaN,0.2430,NaN,0.221,NaN,NaN,NaN,2.700,0.157,0.159,NaN,0.784700,NaN,1.1847,NaN,-0.02220,NaN,-0.00107,NaN,-0.0225,NaN,0.1160,0.105,-0.619,NaN,-1.260,NaN,0.696,0.0521,1.070,NaN,NaN,0.498,0.175,NaN,0.1750,NaN,0.14550,0.0957,0.13535,-0.033150,-0.3720,0.888,NaN,NaN,NaN,NaN,-0.109,NaN,1.120,NaN,NaN,"['Missense_Mutation', 'Missense_Mutation']","['p.G13D', 'p.G12S']"
C3L-00032,NaN,NaN,0.7320,0.00669,NaN,NaN,0.0316,NaN,NaN,-0.0878,NaN,0.635,NaN,0.00669,NaN,NaN,0.7940,NaN,NaN,NaN,NaN,NaN,NaN,-0.422,NaN,0.155,0.0904,NaN,0.0493,-0.3110,NaN,-0.4210,NaN,NaN,NaN,0.163,NaN,0.209,NaN,-0.414,-0.4980,NaN,NaN,0.2260,NaN,NaN,0.0830,NaN,NaN,-0.0025,NaN,0.2010,NaN,-1.400,-0.449,-0.619,NaN,0.165,NaN,NaN,NaN,NaN,0.457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1750,NaN,0.4170,-0.828,-0.0904,NaN,-0.104,NaN,NaN,-0.433,NaN,['Missense_Mutation'],['p.G12C'

In [82]:
hotspot = phos_Kras[(phos_Kras.KRAS_Location.str.contains('G12'))| 
                    phos_Kras.KRAS_Location.str.contains('G13')| 
                    phos_Kras.KRAS_Location.str.contains('Q61')]
wt = phos_Kras[(phos_Kras.KRAS_Mutation.str.contains('Wildtype'))]
hotspot_wt = pd.concat([hotspot, wt])

hotspot_wt= hotspot_wt.replace(r', \'Missense_Mutation\'', "", regex = True)
cols = hotspot_wt.columns.to_list()
cols.remove("KRAS_Mutation")
cols.remove("KRAS_Location")
Kras_phos_pval = u.wrap_ttest(hotspot_wt, 'KRAS_Mutation', cols,return_all=True, pval_return_corrected= True, correction_method= "FDR_bh")
Kras_phos_pval

,Comparison,P_Value
0,RAF1_phosphoproteomics_T330,0.001994
1,RAF1_phosphoproteomics_S29,0.061205
2,PTPN11_phosphoproteomics_S562,0.101354
3,EGFR_phosphoproteomics_S1026,0.166397
4,RAF1_phosphoproteomics_T234,0.166397
5,EGFR_phosphoproteomics_T693,0.166397
6,MAPK1_phosphoproteomics_Y187,0.166397
7,PTPN11_phosphoproteomics_T568,0.176530
8,EGFR_phosphoproteomics_S991,0.207950
9,MAPK1_phosphoproteomics_T185,0.221486


# Colon

In [83]:
phos = col.join_omics_to_mutations(omics_df_name = "phosphoproteomics",mutations_genes="KRAS", tissue_type="tumor")
pd.set_option('display.max_columns', None)
genefilter = phos.columns.get_level_values("Name").isin(phospho_sites)
phos = phos[phos.columns[genefilter]]
#phos = u.reduce_multiindex(df=phos, levels_to_drop=["Database_ID"])
phos_Kras = cptac.utils.reduce_multiindex(phos, flatten=True)
phos_Kras['KRAS_Location'] = phos_Kras['KRAS_Location'].astype(str)
phos_Kras['KRAS_Mutation'] = phos_Kras['KRAS_Mutation'].astype(str)
phos_Kras.head()

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 72 samples for the KRAS gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 1)
cptac warning: phosphoproteomics data was not found for the following samples, so phosphoproteomics data columns were filled with NaN for these samples: 01CO001, 05CO005, 05CO007, 05CO014, 05CO055, 11CO059, 11CO060, 14CO002, 16CO012, 24CO005 (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 1)


Name,EGFR_phosphoproteomics_S1026_P00533,EGFR_phosphoproteomics_S1039_P00533,EGFR_phosphoproteomics_S1042_P00533,EGFR_phosphoproteomics_S1045_P00533,EGFR_phosphoproteomics_S1057_P00533,EGFR_phosphoproteomics_S1064_P00533,EGFR_phosphoproteomics_S1071_P00533,EGFR_phosphoproteomics_S1081_P00533,EGFR_phosphoproteomics_S1104_P00533,EGFR_phosphoproteomics_S1166_P00533,EGFR_phosphoproteomics_S991_P00533,EGFR_phosphoproteomics_T1041_P00533,EGFR_phosphoproteomics_T693_P00533,EGFR_phosphoproteomics_Y1092_P00533,EGFR_phosphoproteomics_Y1172_P00533,EGFR_phosphoproteomics_Y1197_P00533,MAP3K1_phosphoproteomics_S1026_Q13233,MAP3K1_phosphoproteomics_S21_Q13233,MAP3K1_phosphoproteomics_S250_Q13233,MAP3K1_phosphoproteomics_S252_Q13233,MAP3K1_phosphoproteomics_S292_Q13233,MAP3K1_phosphoproteomics_S923_Q13233,MAP3K1_phosphoproteomics_T20_Q13233,MAPK1_phosphoproteomics_T181_P28482,MAPK1_phosphoproteomics_T185_P28482,MAPK1_phosphoproteomics_Y187_P28482,PTPN11_phosphoproteomics_S562_Q06124,PTPN11_phosphoproteomics_T568_Q06124,PTPN11_phosphoproteomics_Y546_Q06124,PTPN11_phosphoproteomics_Y584_Q06124,PTPN11_phosphoproteomics_Y62_Q06124,RAF1_phosphoproteomics_S259_P04049,RAF1_phosphoproteomics_S29_P04049,RAF1_phosphoproteomics_S296_P04049,RAF1_phosphoproteomics_S301_P04049,RAF1_phosphoproteomics_S43_P04049,RAF1_phosphoproteomics_S621_P04049,RAF1_phosphoproteomics_T234_P04049,RAF1_phosphoproteomics_T260_P04049,RAF1_phosphoproteomics_T638_P04049,KRAS_Mutation,KRAS_Location
Patient_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
01CO001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['Wildtype_Tumor'],['No_mutation']
01CO005,NaN,NaN,-0.104,NaN,NaN,NaN,-0.762,NaN,NaN,-1.282,NaN,NaN,-1.499,NaN,NaN,-1.002,NaN,NaN,-0.039,-0.349,NaN,NaN,-0.512,-2.571,-1.359,-1.359,-0.149,NaN,NaN,NaN,0.037,NaN,NaN,NaN,NaN,-0.807,0.066,NaN,-0.749,NaN,['Wildtype_Tumor'],['No_mutation']
01CO006,NaN,0.154,-0.395,-0.395,NaN,NaN,0.030,NaN,NaN,0.150,0.172,-0.395,0.069,0.178,-0.421,NaN,NaN,NaN,1.417,1.417,NaN,NaN,NaN,0.387,NaN,-0.297,-0.091,NaN,-0.827,NaN,-0.041,NaN,NaN,NaN,NaN,0.847,0.102,0.023,0.226,NaN,['Wildtype_Tumor'],['No_mutation']
01CO008,NaN,NaN,NaN,NaN,NaN,NaN,-0.352,NaN,NaN,0.203,NaN,NaN,-0.422,-0.784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.129,-1.401,-0.551,-0.006,NaN,-1.920,NaN,-0.691,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.544,NaN,['nonsynonymous SNV'],['G12V']
01CO013,NaN,-0.234,-0.432,-0.364,NaN,-0.092,0.347,NaN,NaN,-0.543,0.372,-0.364,-0.068,NaN,NaN,NaN,-0.155,NaN,NaN,NaN,-0.269,NaN,1.613,-1.456,NaN,-1.240,-0.310,NaN,NaN,NaN,NaN,NaN,0.192,NaN,NaN,0.831,-0.313,-1.505,-0.184,NaN,['Wildtype_Tumor'],['No_mutation']


In [73]:
hotspot = phos_Kras[(phos_Kras.KRAS_Location.str.contains('G12'))| 
                    phos_Kras.KRAS_Location.str.contains('G13')| 
                    phos_Kras.KRAS_Location.str.contains('Q61')]
wt = phos_Kras[(phos_Kras.KRAS_Mutation.str.contains('Wildtype'))]
hotspot_wt = pd.concat([hotspot, wt])

#hotspot_wt= hotspot_wt.replace(r', \'Missense_Mutation\'', "", regex = True)
cols = hotspot_wt.columns.to_list()
cols.remove("KRAS_Mutation")
cols.remove("KRAS_Location")
Kras_phos_pval = u.wrap_ttest(hotspot_wt, 'KRAS_Mutation', cols,return_all=True, pval_return_corrected= True, correction_method= "FDR_bh")
Kras_phos_pval

,Comparison,P_Value
0,PTPN11_phosphoproteomics_Y62_Q06124,0.042786
1,PTPN11_phosphoproteomics_Y546_Q06124,0.223459
2,MAP3K1_phosphoproteomics_S21_Q13233,0.709738
3,MAP3K1_phosphoproteomics_S250_Q13233,0.709738
4,MAP3K1_phosphoproteomics_S252_Q13233,0.709738
5,MAP3K1_phosphoproteomics_T20_Q13233,0.709738
6,MAPK1_phosphoproteomics_T181_P28482,0.709738
7,MAPK1_phosphoproteomics_T185_P28482,0.709738
8,MAPK1_phosphoproteomics_Y187_P28482,0.709738
9,PTPN11_phosphoproteomics_S562_Q06124,0.709738
